# Chapter 4: Deep Learning

**Welcome to Chapter 4**. This notebook contains the listings for Chapter 4, which explains the fundamentals of deep learning.

# Listing 4-1 A Simply PyTorch Model
This listing implements a subset of the general skeleton for the end-to-end lifecycle of a PyTorch project, which includes loading and preparing data, defining or loading a model, specifying the loss function and optimizer, training with validation, evaluating performance, and saving the model for later use. This code demonstrates the essential stages of a working PyTorch program without including optional steps such as inference pipelines or model deployment.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from pathlib import Path

# 1. Load and prepare data
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('.', train=True, download=True, transform=transform),
    batch_size=64,
    shuffle=True
)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('.', train=False, transform=transform),
    batch_size=1000,
    shuffle=False
)

# 2. Define or load model
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(28 * 28, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)

    def forward(self, x):
        x = x.view(-1, 28 * 28)   # flatten 28x28 → 784
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return self.fc3(x)

model = Net()

# 3. Specify loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 4. Train model
for epoch in range(12):
    model.train()
    running_loss = 0.0

    for data, target in train_loader:
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    avg_train_loss = running_loss / len(train_loader)

    # 5. Validate model during training
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            loss = criterion(output, target)
            val_loss += loss.item()
            preds = output.argmax(dim=1)
            correct += (preds == target).sum().item()
            total += target.size(0)

    avg_val_loss = val_loss / len(test_loader)
    val_accuracy = correct / total * 100

    print(
        f"Epoch {epoch + 1}: "
        f"train loss={avg_train_loss:.4f}, "
        f"val loss={avg_val_loss:.4f}, "
        f"val acc={val_accuracy:.2f}%"
    )

# 6. Evaluate and test (final pass)
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for data, target in test_loader:
        output = model(data)
        preds = output.argmax(dim=1)
        correct += (preds == target).sum().item()
        total += target.size(0)

test_accuracy = correct / total * 100
print(f"Final Test Accuracy: {test_accuracy:.2f}%")

# 7. Save model
save_path = Path("mnist_model.pt").resolve()
torch.save(model.state_dict(), save_path)
print("Model saved to " + str(save_path))


### Listing 4.2 - Using the Trained Model to Predict Custom Digits
This code reloads the saved model and uses it to classify new images. Because the model is already trained, the process is straightforward: load the stored weights, prepare the input images using the same preprocessing steps as during training, and then make predictions on the new images.


In [ ]:
# Minimal MNIST inference in Colab

import io
import torch
import torch.nn as nn
import torch.nn.functional as F
from PIL import Image
from google.colab import files
import torchvision.transforms as T

# 1) Model (same architecture as training)
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(28*28, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)
    def forward(self, x):
        x = x.view(-1, 28*28)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return self.fc3(x)

# 2) Load trained weights
model = Net()
model.load_state_dict(torch.load("/content/mnist_model.pt", map_location="cpu"))
model.eval()

# 3) Preprocessing (match MNIST: 1×28×28 + same normalization)
preprocess = T.Compose([
    T.Grayscale(),          # ensure 1 channel, since MNIST is grayscale
    T.Resize((28, 28)),
    T.ToTensor(),
    T.Normalize((0.1307,), (0.3081,))
])

# 4) Upload image(s) and predict
@torch.no_grad()
def predict(name, content):
    img = Image.open(io.BytesIO(content))
    x = preprocess(img).unsqueeze(0)      # [1,1,28,28]
    pred = model(x).argmax(dim=1).item()
    print(f"The predicted digit for {name} is: {pred}")

print("Upload MNIST-like digit image(s):")
for fname, content in files.upload().items():
    predict(fname, content)
